<a href="https://colab.research.google.com/github/ArtyomShalev/clion-test/blob/master/applied_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
DATA_URL = "http://az.lib.ru/n/nekrasow_n_a/text_1840_pevitza.shtml"

In [88]:
! pip install -q PyYaml==5.3.1
! pip install -q rnnmorph==0.4.0
! pip install -q nltk==3.2.5

In [89]:
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language='ru')

In [90]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset())

In [91]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Некрасов Николай Алексеевич. Певица</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/n/nekrasow_n_a/>Некрасов Николай Алексеевич</a><br>\r\nПевица</h2>\r\n\r'

In [92]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

for script in soup(["script", "style"]):
  script.extract()

cleaned_text = soup.get_text()

In [93]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Некрасов Николай Алексеевич. Певица\n\n\n\nНекрасов Николай Алексеевич\r\nПевица\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Пом'

In [94]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" %len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 911 'sentences'"

In [95]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences")]

sentences: 100%|██████████| 911/911 [00:00<00:00, 96247.13it/s]


In [96]:
len(predictions)

911

In [97]:
only_letter_tokens = []
for i in range(len(predictions)):
  for word in predictions[i]:
    if (word.isalpha()):
      only_letter_tokens.append(word)

In [98]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)

15474

In [99]:
len(only_letter_tokens)

11685

In [100]:
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
#stopwords.words("russian")[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
fdist = FreqDist(only_letter_tokens)
number_of_words = 100

In [102]:
number_of_intersections = 0
most_freq_words = []
for i in range(number_of_words):
  most_freq_words.append(fdist.most_common(number_of_words)[i][0])

intersec = list(set(most_freq_words) & STOPWORDS)
ans = ( number_of_words - len(intersec) ) / number_of_words
print(ans)

0.54


In [104]:
sum = 0
for i in range(number_of_words):
  if (fdist.most_common(number_of_words)[i][1] > 20):
    sum += 1
print(sum)

76
